In [1]:
import torch
import numpy as np

m = torch.Tensor([2.2,4.0],)
print(m)

tensor([2.2000, 4.0000])


In [2]:
## Initialize policy to be evaluated
class Agent():
    def __init__(self, gamma=0.99):
        #discount factor
        self.gamma = gamma
        #estimates of states' values
        self.V = {}
        #state/action spaces
        self.spaces = {
            #possible sums of cards
            "sum":[i for i in range(4,22)],
            #possible cards in dealer's hand
            "dealer_show_card":[i+1 for i in range(10)],
            #useable ace?
            "ace_eleven":[False,True],
            #hit (or stay)?
            "hit":[0,1]
        }
        #combinations of parameters
        self.state_space = []
        #returns
        self.reward = {}
        #has agent visited state before?
        self.states_visited = {}
        #states already encountered/returns received
        self.memory = []
        
        self.init_vals()
        
    def init_vals(self):
        for total in self.spaces["sum"]:
            for card in self.spaces["dealer_show_card"]:
                for ace in self.spaces["ace_eleven"]:
                    self.V[(total, card, ace)] = 0
                    self.reward[(total, card, ace)] = []
                    self.states_visited[(total, card, ace)] = 0
                    self.state_space.append((total, card, ace))
                    
    def policy(self, state):
        total, _, _ = state
        action = 0 if total >= 20 else 1
        return action
        
    def update_V(self):
        for idt, (state, _) in enumerate(self.memory):
            G = 0
            if !states_visited(state)
        return q

In [4]:
## Initialize value function arbitrarily
# def calculate_episode_return(q,gamma,discount_reward,t):
#     t += 1
#     episode_return = q + (gamma**t * discount_reward)
#     return episode_return
def calculate_episode_return(q,gamma,n,t=0):
    episode_return = if (t != n) q[t+1] + (gamma**t)*(calculate_episode_return(q,gamma,n,t+1)) else 0
    return episode_return